In [ ]:
# default_exp core.transfer_model

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Transfer learning temporal model 
> Trains the temporal model from the Video Pose3D checkpoint made for inference. 

In [ ]:
#export
import time
import numpy as np
import torch

from torch import optim
from runningpose.core.runningpose_dataset import RunningposeDataset
from runningpose.core.model import TemporalModel
from runningpose.core.generators import ChunkedGenerator, UnchunkedGenerator

#### Load Data
We need our 2D data to match the 3D ground truth data. 

In [ ]:
#export
# print('Loading dataset...')
# dataset_path = ''
# dataset = RunningposeDataset(dataset_path)

# Convert to 3D camera coordinates 
# TODO: Maybe just do this in format qtmdata. 

In [ ]:
#export
print('Loading 2D detections...')
keypoints = np.load('data_2d_custom_tindra.npz', allow_pickle=True)
keypoints_metadata = keypoints['metadata'].item()
keypoints_symmetry = keypoints_metadata['keypoints_symmetry']
kps_left, kps_right = list(keypoints_symmetry[0]), list(keypoints_symmetry[1])
# joints_left, joints_right = list(dataset.skeleton().joints_left()), list(dataset.skeleton().joints_right())
keypoints = keypoints['positions_2d'].item()
print(keypoints.keys())
print(keypoints['Tindra_dynamic0001_Miqus_16_23604.avi'].keys())
print(keypoints['Tindra_dynamic0001_Miqus_16_23604.avi']['custom'][0].shape)

Loading 2D detections...
dict_keys(['Tindra_dynamic0001_Miqus_16_23604.avi', 'Tindra_dynamic0001_Miqus_15_21386.avi'])
dict_keys(['custom'])
(694, 17, 2)


We can see that we have a dictionary with all the videos and then a 
custom dictionary with a list with an array with (frames, keypoints, dim) 

#### Define the model
> Load the temporal model trained model for generating 3D predictions.

In [ ]:
#export
# Load checkpoint
print('Loading checkpoint')
checkpoint = torch.load('pretrained_h36m_detectron_coco.bin', 
                        map_location=lambda storage,
                        loc: storage)
print('This model was trained for {} epochs'.format(checkpoint['epoch']))
checkpoint.keys()

Loading checkpoint
This model was trained for 80 epochs


dict_keys(['epoch', 'lr', 'model_pos'])

In [ ]:
#export
# Hyperparameters
num_joints_in = 17 # COCO
in_features = 2 # dimension of in joints
num_joints_out = 20 # runningpose
filter_widths = [3, 3, 3, 3, 3] # just as in inference  
causal = False # No real time predictions 
dropout = 0.25 # default
channels = 1024 # default
lr = 0.001 # default
lr_decay = 0.95 # default
batch_size = 64
chunk_length = 1

model_run_train = TemporalModel(
    num_joints_in, in_features, num_joints_out, filter_widths, causal, 
    dropout, channels
)
if torch.cuda.is_available():
    model_run_train = model_run_train.cuda()

model_run_train.load_state_dict(checkpoint['model_pos'])

<All keys matched successfully>

In [ ]:
#export
# Calculate padding based on receptive field
receptive_field = model_run_train.receptive_field()
print('INFO: Receptive field: {} frames'.format(receptive_field))
pad = (receptive_field - 1) // 2 # Padding on each side

INFO: Receptive field: 243 frames


In [ ]:
#export
# Optimizer
optimizer = optim.Adam(model_run_train.parameters(), lr=lr, amsgrad=True)
scaler = torch.cuda.amp.GradScaler()

# Initialize loss
losses_3d_train = []
losses_3d_train_eval = []
losses_3d_valid = []

epoch = 0
# Using batch norm momentum
initial_momentum = 0.1
final_momentum = 0.001

In [ ]:
#export
train_generator = ChunkedGenerator(
    batch_size, cameras=None, poses_3d=None, poses_2d=None, 
    chunk_length=chunk_length, shuffle=True, augment=False, kps_left=kps_left, 
    kps_right=kps_right, joints_left=None, joints_right=None
)

train_generator_eval = UnchunkedGenerator(
    cameras=None, poses_3d=None, poses_2d=None, pad=pad, augment=False
)
print('INFO: Training on {} frames'.format(train_generator_eval.num_frames()))

TypeError: object of type 'NoneType' has no len()